In [1]:
# add default values for parameters here

In [2]:
# Parameters
product = "notebooks/tsne.ipynb"

In [4]:
import __path__

In [5]:
from config import MAIN_EXPERIMENT


AFFILIATION_TH = MAIN_EXPERIMENT.affiliation_theshold
PREPROCESSED_FILE = MAIN_EXPERIMENT.preprocessed_file
MAIN_MODEL = MAIN_EXPERIMENT.main_model
PICTURES = MAIN_EXPERIMENT.pictures
GROUPS = MAIN_EXPERIMENT.groups
TSNE_ = MAIN_EXPERIMENT.tsne

In [6]:
import json


with open(PREPROCESSED_FILE, 'r') as f:
    preprocessed = json.load(f)

In [7]:
from gensim.models import TfidfModel

from src.utils import load


lda_model, dictionary, corpus = load(MAIN_MODEL)
tfidf = TfidfModel(corpus, id2word=dictionary)

In [8]:
import os
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd

topics = []
for i, row_list in enumerate(lda_model[corpus][:]):
    proba = lda_model.get_document_topics(corpus[i], minimum_probability=0.0)
    probas = [p[1] for p in proba]
    topics.append([np.argmax(probas), *probas])

df = pd.DataFrame(topics).sort_values(by=0).reset_index().drop(columns='index')
topics_count = df.iloc[:,0].nunique()
df

tsne_lda = None
if not os.path.isfile(TSNE_):
    tsne_model = TSNE(n_components=2, verbose=0, random_state=0, angle=.99)
    tsne_lda = tsne_model.fit_transform(df.iloc[:,1:-1].values)
    np.save(TSNE_, tsne_lda)
else:
    tsne_lda = np.load(TSNE_)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0,0.603515,0.018022,0.018022,0.018022,0.018022,0.018022,0.018022,0.018022,0.018022,...,0.018022,0.018022,0.018022,0.018022,0.018022,0.018022,0.018022,0.018022,0.018022,0.018022
1,0,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,...,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478
2,0,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,...,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478
3,0,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,...,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478
4,0,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,...,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49764,22,0.013627,0.013627,0.013627,0.013627,0.013627,0.013627,0.013627,0.013627,0.013627,...,0.013627,0.013627,0.013627,0.013627,0.013627,0.013627,0.013627,0.013627,0.013627,0.700203
49765,22,0.012670,0.012670,0.012670,0.012670,0.012670,0.012670,0.012670,0.012670,0.012670,...,0.012670,0.012670,0.012670,0.012670,0.012670,0.012670,0.012670,0.012670,0.012670,0.721265
49766,22,0.010219,0.010219,0.010219,0.010219,0.010219,0.010219,0.010219,0.010219,0.010219,...,0.010219,0.010219,0.010219,0.010219,0.010219,0.010219,0.010219,0.010219,0.010219,0.775176
49767,22,0.013627,0.013627,0.013627,0.013627,0.013627,0.013627,0.013627,0.013627,0.013627,...,0.013627,0.013627,0.013627,0.013627,0.013627,0.013627,0.013627,0.013627,0.013627,0.700203


In [9]:
import random
import plotly.graph_objects as go

from src.utils import rgba

cs = lambda n: list(map(lambda i: f"#{random.randint(0x99, 0xdd):02X}{random.randint(0x99, 0xdd):02X}{random.randint(0x99, 0xdd):02X}", range(n)))


fig = go.Figure()

for i in range(topics_count):
    if not topics_count:
        continue
    coords = df.index[df[0] == i].tolist()
    fig.add_trace(
        go.Scatter(
            name=f'Тематика {i}',
            x=tsne_lda[coords[0]:coords[-1]+1,0], 
            y=tsne_lda[coords[0]:coords[-1]+1,1], 
            mode='markers'))

fig.update_layout(
    font=dict(family='Times New Roman', size=20, color='black'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(pad=10),
    colorway=[rgba(c, .15) for c in cs(topics_count)],
    height=800,
    width=800)

fig.update_xaxes(
    title=dict(
        font=dict(family='Times New Roman', size=25, color='black'),
        text='PC1'), 
    zerolinecolor='lightgrey',
    gridcolor='lightgrey',
    tickformat='000', 
    showgrid=True, 
    tickangle=0,
    ticklen=10)

fig.update_yaxes( 
    title=dict(
        font=dict(family='Times New Roman', size=25, color='black'),
        text='PC2'), 
    zerolinecolor='lightgrey',
    gridcolor='lightgrey',
    tickformat='000', 
    showgrid=True, 
    tickangle=0,
    ticklen=10)

fig.update_layout(showlegend=False)

In [10]:
fig.write_image(f'{PICTURES}/tsne.png')